In [2]:
import pandas as pd
import numpy as np
import sklearn
import math 
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

In [23]:
train_url = "./modelsData/trafficWeatherAccident.csv"
data = pd.read_csv(train_url , header = 0)

In [24]:
dfDummies1 = pd.get_dummies(data['zona'], prefix = 'zona')
dfDummies2 = pd.get_dummies(data['subZona'], prefix = 'subZona')
dfDummies3 = np.sin(data['hora']) 
dfDummies4 = pd.get_dummies(data['linkName'], prefix = 'linkName') 
dfDummies5 = pd.get_dummies(data['descripcionClima'], prefix = 'clima') 

In [25]:
df = pd.concat([data, dfDummies1,dfDummies2,dfDummies3,dfDummies4,dfDummies5], axis=1)


In [26]:
aux = pd.DataFrame()


sumHM = df["muertos"] + df ["heridos"]
y_model_1 = sumHM > 0 #zona peligrosa  



y_model_2 = df["numeroVehiculos"] >= df["numeroVehiculos"].mean() # zona de alto trafico



aux["Zona flujo rapido"] = df["velocidadProm"] >= 55
aux["Zona flujo lento"] = df["velocidadProm"] <= 30
aux["Zona flujo normal"] = np.where((df["velocidadProm"] < 55) & (df["velocidadProm"] > 30), 1, 0)


aux = aux*1

In [30]:
aux.idxmax(1).head()

0    Zona flujo normal
1    Zona flujo normal
2    Zona flujo rapido
3    Zona flujo normal
4     Zona flujo lento
dtype: object

In [39]:
df = df.drop(["zona","subZona", "hora","descripcionClima","encodedPolyLine","latitude","longitude","linkName","date"], axis = 1)  


KeyError: "['zona' 'subZona' 'hora' 'descripcionClima' 'encodedPolyLine' 'latitude'\n 'longitude' 'linkName' 'date'] not found in axis"

In [40]:
X_train, X_test, y_train, y_test = train_test_split(df, y_model_1, test_size=.2, random_state=42)

In [42]:
X_test.to_csv("./datasetsModels/datasetClasificacionZona.csv")

## Primer modelo zona peligrosa

In [28]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(df.drop(["heridos","muertos"],axis = 1), y_model_1, test_size=.2, random_state=42)

In [38]:
X_test_1.to_csv("./datasetsModels/testDatasetModelZonaPeligrosa.csv")

In [11]:
xgb_1 = XGBClassifier()
xgb_1.fit(X_train_1,y_train_1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [12]:
print("training accuracy: ",xgb_1.score(X_train_1,y_train_1),"test accuracy: ",xgb_1.score(X_test_1,y_test_1))

training accuracy:  0.7856707598255577 test accuracy:  0.7872238105903898


In [14]:
clf_1 = LogisticRegression(max_iter=500)
clf_1.fit(X_train_1, y_train_1);

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [13]:
print("training accuracy: ",clf_1.score(X_train_1,y_train_1),"test accuracy: ",clf_1.score(X_test_1,y_test_1))

training accuracy:  0.7850391638888523 test accuracy:  0.786446461745214


In [15]:
print("training accuracy: ",clf_1.score(X_train_1,y_train_1),"test accuracy: ",clf_1.score(X_test_1,y_test_1))

training accuracy:  0.7850391638888523 test accuracy:  0.786446461745214


## Segundo modelo zona altamente concurrida

In [31]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(df.drop(["numeroVehiculos"],axis = 1), y_model_2, test_size=.2, random_state=42)

In [37]:
X_test_2.to_csv("./datasetsModels/testDatasetModelAltamenteConcurrida.csv")

In [14]:
xgb_2 = XGBClassifier()
xgb_2.fit(X_train_2,y_train_2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [15]:
print("training accuracy: ",xgb_2.score(X_train_2,y_train_2),"test accuracy: ",xgb_2.score(X_test_2,y_test_2))

training accuracy:  0.9463810458635818 test accuracy:  0.9462608203006627


In [46]:
clf_2 = LogisticRegression()
clf_2.fit(X_train_2, y_train_2);

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [19]:
print("training accuracy: ",clf_2.score(X_train_2,y_train_2),"test accuracy: ",clf_2.score(X_test_2,y_test_2))

training accuracy:  0.9455732288962964 test accuracy:  0.9456382824543142


In [47]:
import pickle

In [48]:
pickle.dump(clf_2, open("clasificacionZonas.model", 'wb'))

## Tercer modelo tipo de flujo en la zona

In [18]:
tipos_flujos = aux.idxmax(1)
leZonas = preprocessing.LabelEncoder()
leZonas.fit(tipos_flujos)
y_model_3 = leZonas.transform(tipos_flujos)


In [33]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(df.drop(["velocidadProm"],axis = 1), y_model_3, test_size=.2, random_state=42)

In [36]:
X_test_3.to_csv("./datasetsModels/testDatasetModelTipoDeFlujo.csv")

In [19]:
xgb_3 = XGBClassifier()
xgb_3.fit(X_train_3,y_train_3)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [20]:
print("training accuracy: ",xgb_3.score(X_train_3,y_train_3),"test accuracy: ",xgb_3.score(X_test_3,y_test_3))

training accuracy:  0.8608026785596649 test accuracy:  0.8604296499295115


In [22]:
clf_3 = LogisticRegression(max_iter=2000)
clf_3.fit(X_train_3, y_train_3);

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [21]:
print("training accuracy: ",clf_3.score(X_train_3,y_train_3),"test accuracy: ",clf_3.score(X_test_3,y_test_3))

training accuracy:  0.8792457739891171 test accuracy:  0.8784371335590719


In [11]:
print("finished")

finished


In [51]:
modeloPrueba = pickle.load(open("clasificacionZonas.model","rb"))

In [55]:
X_train_3.iloc[:,1]

432780     4
412515     5
1511441    1
771710     6
943698     2
187620     3
787370     7
248218     2
196824     7
929223     7
1249429    2
434553     3
887715     4
483468     7
642027     2
1496061    3
717483     7
1271380    6
1112998    7
268697     3
1429527    6
734986     7
842620     5
365031     2
409458     7
986972     3
1019637    4
922657     3
800351     3
866930     7
          ..
989436     4
1470485    4
1396025    7
184779     5
1262752    6
1284372    4
103355     7
791743     6
1247617    6
327069     3
1370455    6
787201     2
1113396    4
329365     5
41090      1
278167     5
1239911    6
175203     2
912756     6
1136074    3
999890     6
137337     6
1103462    5
732180     3
110268     5
259178     6
1414414    7
131932     3
671155     6
121958     1
Name: diaSemana, Length: 1214384, dtype: int64

In [57]:
modeloPrueba.predict_proba(X_train_3.sample(1).values)

array([[9.99778685e-01, 2.21314931e-04]])

In [59]:
clf_3.predict_proba(X_train_3.sample(1).values)

array([[9.99987260e-01, 1.27404458e-05, 6.89673036e-40]])